# Testing on the dataset

## Initialization

In [2]:

from datasets import load_dataset


ds = load_dataset("CharlyR/vtikz", "tikz", split="test")
ds = ds.select_columns(["id","type","instruction","code","image_solution","image_input","code_solution"])


In [6]:
from openai import OpenAI
import os
from google import genai
from google.genai import types as genTypes
from vif.falcon.edition import OracleEditionModule
from vif.falcon.falcon import Falcon
from vif.falcon.oracle.guided_oracle.guided_code_oracle import OracleGuidedCodeModule
from vif.utils.renderer.tex_renderer import TexRenderer

gclient = genai.Client(
    api_key=os.environ.get("GOOGLE_API_KEY"),
    http_options=genTypes.HttpOptions(api_version="v1alpha"),
)

oclient = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.environ.get("OPENROUTER_API_KEY"),
)


oracle_module = OracleGuidedCodeModule(
    model="google/gemini-2.5-flash-lite",
    temperature=0.3,
    client=oclient,
    visual_client=gclient,
    visual_model="gemini-2.5-flash"
)
edition_module = OracleEditionModule(
    model="google/gemini-2.5-flash-lite",
    temperature=0.3,
    client=oclient,
)

agent = Falcon(
    code_renderer=TexRenderer().from_string_to_image,
    oracle_module=oracle_module,
    edition_module=edition_module,
    observe=True,
    observe_folder="/home/creux/Documents/AI/VIFagent/.tmp/observe",
    #existing_observe_checkpoint="/home/creux/Documents/AI/VIFagent/.tmp/observe/2008-10:43:13.pickle"
)

2025-08-21 12:45:19.511 | WARNING  | vif.falcon.falcon:__init__:57 - The observe parameter is activated, dict will be stored at /home/creux/Documents/AI/VIFagent/.tmp/observe/2108-12:45:19.pickle


#### Manual Tests on one example

In [7]:
agent.apply_instruction(ds[0]["code"],ds[0]["instruction"],ds[0]["id"])

2025-08-21 12:45:23.574 | INFO     | vif.falcon.falcon:apply_instruction:91 - Creating the oracle
2025-08-21 12:45:23.575 | INFO     | vif.falcon.oracle.guided_oracle.guided_code_oracle:get_oracle:109 - Creating Oracle for instruction Switch the color of the circle and the triangle
2025-08-21 12:45:25.493 | INFO     | vif.falcon.oracle.guided_oracle.guided_code_oracle:get_oracle:124 - Oracle created: def test_valid_customization() -> bool:
    return color("circle", "red") & color("triangle", "green")

2025-08-21 12:45:25.818 | INFO     | vif.falcon.edition:customize:117 - Sending initial message
2025-08-21 12:45:27.541 | INFO     | vif.falcon.edition:customize:127 - LLM response:ChatCompletion(id='gen-1755773125-2IsoJJUetLu10XqQuI6l', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='```\n[\n  {"start": 8, "end": 9, "content": "  \\\\node[regular polygon, regular polygon sides=3, minimum size=3cm, fill=green!30] at (0,0) {};"},\n  {"s

'\\documentclass[tikz,border=5]{standalone}\n\\usepackage{tikz}\n\\usetikzlibrary{shapes}\n\n\\begin{document}\n\n\\begin{tikzpicture}\n  \\node[regular polygon, regular polygon sides=3, minimum size=3cm, fill=green!30] at (0,0) {};\n  \\node[circle, minimum size=2cm, fill=red!30] at (3,0.25) {};\n  \\node[rectangle, minimum size=2cm, fill=blue!30] at (1.5,2.5) {};\n\\end{tikzpicture}\n\\end{document}'

In [8]:
import pickle
with open("/home/creux/Documents/AI/VIFagent/.tmp/observe/2108-12:45:19.pickle","rb") as pkds:
    dict_ds = pickle.load(pkds)

In [9]:
dict_ds

[{'id': 'rgbstc_switch_col_ct',
  'instruction': 'Switch the color of the circle and the triangle',
  'original_code': '\\documentclass[tikz,border=5]{standalone}\n\\usepackage{tikz}\n\\usetikzlibrary{shapes}\n\n\\begin{document}\n\n\\begin{tikzpicture}\n  \\node[regular polygon, regular polygon sides=3, minimum size=3cm, fill=red!30] at (0,0) {};\n  \\node[circle, minimum size=2cm, fill=green!30] at (3,0.25) {};\n  \\node[rectangle, minimum size=2cm, fill=blue!30] at (1.5,2.5) {};\n\\end{tikzpicture}\n\\end{document}',
  'oracle_code': 'def test_valid_customization() -> bool:\n    return color("circle", "red") & color("triangle", "green")\n',
  'original_image': <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=446x363>,
  'turn': 0,
  'custom_image': <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=446x363>,
  'custom_code': '\\documentclass[tikz,border=5]{standalone}\n\\usepackage{tikz}\n\\usetikzlibrary{shapes}\n\n\\begin{document}\n\n\\begin{tikzpicture}\n  \\node[regular p

### Executing on the dataset

In [ ]:
def run_apply_instruction(row):
    agent.apply_instruction(row["code"],row["instruction"],row["id"])


ds.map(run_apply_instruction)
    